 This method involves using the original graph, constructed from the provided edges. We generate a graph vector for each node pair by employing the metrics defined in paper. These vectors are used to train the classifier, which is then employed to predict the labels for new incoming edges.

## Offline

Create Graph

In [1]:
#Import train dataset
import pandas as pd

df_edges_all=pd.read_csv('train.csv')
df_edges_all_test=pd.read_csv('test.csv')


df_edges= df_edges_all

import pickle 
sparsevectors={}
with open('text_vectors.pkl', 'rb') as f:
    sparsevectors = pickle.load(f)

# The existing graph
import networkx as nx

G= nx.Graph()
#add nodes
nodestest=pd.unique(df_edges_all_test[['id1', 'id2']].values.ravel())
nodes = pd.unique(df_edges_all[['id1', 'id2']].values.ravel())
G.add_nodes_from(nodes)
G.add_nodes_from(nodestest)

#add edges
edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==1].itertuples()]
G.add_edges_from(edges)

# keeping all the negative edges so we can compare results
no_edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==0].itertuples()]

Define graph metrics 

In [2]:
import math
# Adding methods proposed in the https://link.springer.com/content/pdf/10.1038/s41598-019-57304-y.pdf
def shortest_path(G:nx.Graph,source:int,target:int)->float:
    try:
        p=nx.shortest_path(G,source=source,target=target)
        return len(p)-1
    except nx.NetworkXNoPath:
        return float("inf")

def distance(G:nx.Graph,source:int,target:int)->float:
   '''
   This method can be changed to any distance
   '''
   return shortest_path(G,source,target)

def closeness_centrality(G:nx.Graph, source:int, target:int)->float:
    dxy = nx.shortest_path(G,source=source,target=target)
    return G.number_of_nodes()/shortest_path(G,source,target)

def common_neighbors(G:nx.Graph, source:int, target:int)->list:
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    return common_neighbors

def CCPA(G:nx.Graph, source:int, target:int,a:float = 0.5)->float:
    '''
    Common Neighbor and Centrality based Parameterized Algorithm
    '''
    return a*closeness_centrality(G,source,target)+(1-a)*len(common_neighbors(G,source,target))

def CND(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor and Distance
    '''
    cn = common_neighbors(G,source,target)
    if len(cn)>0:
        return (len(cn)+1)/2
    else:
        if distance(G,source,target)==0:
            return 0
        return 1/distance(G,source,target)

def PA(G:nx.Graph, source:int, target:int)->float:
    '''
    Preferential Attachment
    '''
    return G.degree[source]*G.degree[target]

def AA(G:nx.Graph, source:int, target:int)->float:
    '''
    Adamic Adar
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / math.log(degree)
    return similarity

def CN(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor
    '''
    return len(common_neighbors(G,source,target))

def SI(G:nx.Graph, source:int, target:int)->float:
    '''
    Sorensen Index
    '''
    if (G.degree[source]+G.degree[target]) ==0:
        return 0
    return 2*CN(G,source,target)/(G.degree[source]+G.degree[target])

def JI(G:nx.Graph, source:int, target:int)->float:
    '''
    Jaccard Index
    '''
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    all_neighbors = set(s_neighbors).union(t_neighbors)
    if len(all_neighbors)==0:
        return 0
    return len(common_neighbors)/len(all_neighbors)

def RA(G:nx.Graph, source:int, target:int)->float:
    '''
    Resource Allocation
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / degree
    return similarity

def HPI(G:nx.Graph, source:int, target:int)->float:
    '''
    Hub Promoted Index
    '''
    if min([G.degree[source],G.degree[target]]) ==0 :
        return 0
    return CN(G,source,target)/min([G.degree[source],G.degree[target]])

def create_metric_vector(tup):
    G=tup[0]
    source= tup[1]
    target= tup[2]
    functions = [CND, PA, AA, CN, SI, JI, RA, HPI]
    outputs = []
    for func in functions:
        output = func(G, source, target)
        outputs.append(output)
    return outputs
from multiprocessing import Pool


def makedatasetapply(df):
    label=df.label
    datadf = df.apply(lambda row: create_metric_vector((G,row['id1'],row['id2'])), axis=1,result_type='expand')
    return datadf,label

Create vector for each pair in training set

In [3]:
from tqdm import tqdm

X,Y=makedatasetapply(df_edges)

Xcopy=X.copy()
Xcopy["label"]=Y
Xcopy.to_csv("simple_vectors_with_labels.csv",index=False)



Train a classifier in vectors derived fromt training setfor

In [4]:
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
model = XGBClassifier()

Xcopy=pd.read_csv("simple_vectors_with_labels.csv")
Y=Xcopy["label"]
X=Xcopy.drop(["label"],axis=1)
model.fit(X, Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# Online

Create vectors using the graph

In [5]:
def makedataset_TEST(df):
    datadf = df.apply(lambda row: create_metric_vector((G,row['id1'],row['id2'])), axis=1,result_type='expand')
    return datadf

X_test=makedataset_TEST(df_edges_all_test)

Produce predictions using trained model

In [6]:
preds = model.predict(X_test)

Store predictions to submission file

In [7]:
dfexpe=pd.DataFrame(zip(df_edges_all_test["id"].values,preds),columns=["id","label"])
dfexpe.to_csv("simple_graph_submission.csv",index=False)